In [2]:
from datasets import load_dataset
from tqdm import tqdm
import json
import jsonlines

/home/yaojiarui/miniconda3/envs/MoE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'

In [4]:
def check_parity(re):
    for item in re['instances']:
        for i, message in enumerate(item['messages']):
            if i % 2:
                assert message['role'] == 'assistant'
            else:
                assert message['role'] == 'user'

def check_parity_ob(re):
    for item in re['instances']:
        for i, message in enumerate(item['messages']):
            if i % 4 == 0:
                assert message['role'] == 'user'
            elif i % 4 == 1:
                assert message['role'] == 'function'
            elif i % 4 == 2:
                assert message['role'] == 'observation'
            else:
                assert message['role'] == 'assistant'

In [5]:
dataset_names = [
    'glaiveai/glaive-function-calling-v2',
    'Salesforce/xlam-function-calling-60k',
    'Gorilla OpenFunctions-v2',
    'Gorilla OpenFunctions-v1',
    'ise-uiuc/Magicoder-OSS-Instruct-75K',
    'RLHFlow/CodeUltraFeedback-standard',
    'codeparrot/apps',
    'meta-math/MetaMathQA',
    # 'TIGER-Lab/MathInstruct',
    'camel-ai/math',
    # 'xinlai/Math-Step-DPO-10K',
    'openai/gsm8k'
]

In [55]:
# process glaiveai/glaive-function-calling-v2
ds = load_dataset('glaiveai/glaive-function-calling-v2')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = item['system'].lstrip('SYSTEM:').strip()
    chat = item['chat']
    instantce = {"conversation_id": i, "system": system, "messages": []}
    user_assistant_pairs = chat.split('USER: ')[1:]
    for pair in user_assistant_pairs:
        turn = pair.split('ASSISTANT: ')
        message = {"role": "user", "content": turn[0].strip()}
        instantce['messages'].append(message)
        # only user and assistant
        assistant_message = ''
        for i in range(1, len(turn)):
            assistant_message += turn[i].strip()
        instantce['messages'].append({"role": "assistant", "content": assistant_message})        

    re['instances'].append(instantce)

with open('data/glaive-function-calling-v2.json', 'w') as f:
    json.dump(re, f, indent=4)

100%|██████████| 112960/112960 [00:03<00:00, 33582.18it/s]


In [54]:
# process glaiveai/glaive-function-calling-v2 in sharegpt form
ds = load_dataset('glaiveai/glaive-function-calling-v2')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = item['system'].lstrip('SYSTEM:').strip()
    chat = item['chat']
    instantce = {"conversation_id": i, "system": system, "messages": []}
    user_assistant_pairs = chat.split('USER: ')[1:]
    for pair in user_assistant_pairs:
        turn = pair.split('ASSISTANT: ')
        message = {"role": "user", "content": turn[0].strip()}
        instantce['messages'].append(message)
        done = False
        for i in range(1, len(turn)):
            try:
                call, ob = turn[i].split('FUNCTION RESPONSE: ')
                instantce['messages'].append({"role": "function", "content": call.strip().lstrip('<functioncall>').rstrip('<|endoftext|>').strip()})
                instantce['messages'].append({"role": "observation", "content": ob.strip()})
                assistant_message = ''
                for j in range(1, len(turn)):
                    if j == i:
                        continue
                    assistant_message += turn[j].strip()
                instantce['messages'].append({"role": "assistant", "content": assistant_message})
                done = True
                break
            except:
                continue

        if not done:
            if len(turn) == 1:
                instantce['messages'].append({"role": "function", "content": ''})
                instantce['messages'].append({"role": "observation", "content": ''})
                instantce['messages'].append({"role": "assistant", "content": ''})
            elif len(turn) == 2:
                instantce['messages'].append({"role": "function", "content": ''})
                instantce['messages'].append({"role": "observation", "content": ''})
                instantce['messages'].append({"role": "assistant", "content": turn[1].strip()})
            else:
                print(turn)
                raise Exception("Invalid chat")

        # if len(turn) == 1:
        #     instantce['messages'].append({"role": "function", "content": ''})
        #     instantce['messages'].append({"role": "observation", "content": ''})
        #     instantce['messages'].append({"role": "assistant", "content": ''})
        # elif len(turn) == 2:
        #     instantce['messages'].append({"role": "function", "content": ''})
        #     instantce['messages'].append({"role": "observation", "content": ''})
        #     instantce['messages'].append({"role": "assistant", "content": turn[1].strip()})
        # elif len(turn) == 3:
        #     try:
        #         call, ob = turn[1].split('FUNCTION RESPONSE: ')
        #         instantce['messages'].append({"role": "function", "content": call.strip().lstrip('<functioncall>').rstrip('<|endoftext|>').strip()})
        #         instantce['messages'].append({"role": "observation", "content": ob.strip()})
        #         instantce['messages'].append({"role": "assistant", "content": turn[2].strip()})
        #     except:
        #         call, ob = turn[2].split('FUNCTION RESPONSE: ')
        #         instantce['messages'].append({"role": "function", "content": call.strip().lstrip('<functioncall>').rstrip('<|endoftext|>').strip()})
        #         instantce['messages'].append({"role": "observation", "content": ob.strip()})
        #         instantce['messages'].append({"role": "assistant", "content": turn[1].strip()})
        # elif len(turn) == 4:
        #     call, ob = turn[2].split('FUNCTION RESPONSE: ')
        #     instantce['messages'].append({"role": "function", "content": call.strip().lstrip('<functioncall>').rstrip('<|endoftext|>').strip()})
        #     instantce['messages'].append({"role": "observation", "content": ob.strip()})
        #     instantce['messages'].append({"role": "assistant", "content": turn[1].strip()+turn[3].strip()})
        # elif len(turn) == 5:
        #     call, ob = turn[1].split('FUNCTION RESPONSE: ')
        #     instantce['messages'].append({"role": "function", "content": call.strip().lstrip('<functioncall>').rstrip('<|endoftext|>').strip()})
        #     instantce['messages'].append({"role": "observation", "content": ob.strip()})
        #     instantce['messages'].append({"role": "assistant", "content": turn[1].strip()+turn[3].strip()})
        # else:
        #     print(turn)
        #     raise Exception("Invalid chat")

        # # only user and assistant
        # assistant_message = ''
        # for i in range(1, len(turn)):
        #     assistant_message += turn[i].strip()
        # instantce['messages'].append({"role": "assistant", "content": assistant_message})        

    re['instances'].append(instantce)

with open('data/glaive-function-calling-v2-sharegpt.json', 'w') as f:
    json.dump(re, f, indent=4)

100%|██████████| 112960/112960 [00:04<00:00, 27512.44it/s]


In [18]:
chat = ds['train'][1]['chat']
pairs = chat.split('USER: ')[1:]

In [29]:
pairs[0].split('ASSISTANT: ')[1].split('FUNCTION RESPONSE: ')

['<functioncall> {"name": "get_news_headlines", "arguments": \'{"country": "United States"}\'} <|endoftext|>\n\n\n',
 '{"headlines": ["Biden announces new vaccine mandates", "Hurricane Ida devastates Louisiana", "Apple unveils new iPhone", "NASA\'s Perseverance rover collects first Mars rock sample"]}\n\n\n']

In [5]:
# process hiyouga/glaive-function-calling-v2-sharegpt
# ds = load_dataset('hiyouga/glaive-function-calling-v2-sharegpt')
# 数据并不都是 {user, assistant}或{user, function, observation, assistant}的格式

# re = {"type": "conversation", "instances": []}
# for i, item in enumerate(tqdm(ds['train'])):
#     instantce = {"conversation_id": i, "system": '', "tools": item['tools'], "messages": []}
#     chat = item['conversations']
#     index = 0
#     done = False
#     while index < len(chat):
#         if chat[index]['from'] == 'human':
#             instantce['messages'].append({"role": "user", "content": chat[index]['value']})
#             if chat[index+1]['from'] == 'gpt':
#                 instantce['messages'].append({"role": "function", "content": ''})
#                 instantce['messages'].append({"role": "observation", "content": ''})
#                 instantce['messages'].append({"role": "assistant", "content": chat[index+1]['value']})
#                 index += 2
#             else:
#                 instantce['messages'].append({"role": "function", "content": chat[index+1]['value']})
#                 instantce['messages'].append({"role": "observation", "content": chat[index+2]['value']})
#                 instantce['messages'].append({"role": "assistant", "content": chat[index+3]['value']})
#                 index += 4
#         else:
#             print(chat)
#             done = True
#             break
#     if done:
#         break
#     re['instances'].append(instantce)

# with open('data/glaive-function-calling-v2-sharegpt.json', 'w') as f:
#     json.dump(re, f, indent=4)

In [35]:
# process Salesforce/xlam-function-calling-60k
ds = load_dataset('Salesforce/xlam-function-calling-60k')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = ''
    messages = [{"role": "user", "content": item['query']}, {"role": "assistant", "content": item['answers']}]
    instantce = {"conversation_id": i, "system": system, "tools": [str(tool) for tool in json.loads(item['tools'])], "messages": messages}
    re['instances'].append(instantce)

with open('data/xlam-function-calling-60k.json', 'w') as f:
    json.dump(re, f, indent=4)

100%|██████████| 60000/60000 [00:03<00:00, 18310.08it/s]


In [24]:
# process gorilla/data/apibench
api_path = 'gorilla/data/api'
huggingface_api = []
with jsonlines.open(f'{api_path}/huggingface_api.jsonl', 'r') as f:
    for line in f:
        huggingface_api.append(line)
tensorflowhub_api = []
with jsonlines.open(f'{api_path}/tensorflowhub_api.jsonl', 'r') as f:
    for line in f:
        tensorflowhub_api.append(line)
torchhub_api = []
with jsonlines.open(f'{api_path}/torchhub_api.jsonl', 'r') as f:
    for line in f:
        torchhub_api.append(line)

apibench_path = 'gorilla/data/apibench'
huggingface_train = []
with jsonlines.open(f'{apibench_path}/huggingface_train.json', 'r') as f:
    for line in f:
        huggingface_train.append(line)
huggingface_eval = []
with jsonlines.open(f'{apibench_path}/huggingface_eval.json', 'r') as f:
    for line in f:
        huggingface_eval.append(line)
tensorflow_train = []
with jsonlines.open(f'{apibench_path}/tensorflow_train.json', 'r') as f:
    for line in f:
        tensorflow_train.append(line)
tensorflow_eval = []
with jsonlines.open(f'{apibench_path}/tensorflow_eval.json', 'r') as f:
    for line in f:
        tensorflow_eval.append(line)
torchhub_train = []
with jsonlines.open(f'{apibench_path}/torchhub_train.json', 'r') as f:
    for line in f:
        torchhub_train.append(line)
torchhub_eval = []
with jsonlines.open(f'{apibench_path}/torchhub_eval.json', 'r') as f:
    for line in f:
        torchhub_eval.append(line)


In [25]:
dss = {'huggingface_train': huggingface_train,
       'huggingface_eval': huggingface_eval,
       'tensorflow_train': tensorflow_train,
       'tensorflow_eval': tensorflow_eval,
       'torchhub_train': torchhub_train,
       'torchhub_eval': torchhub_eval}
def process_api(api_name, ds):
    re = {"type": "conversation", "instances": []}
    for i, item in enumerate(tqdm(dss[ds])):
        system = ''
        turn = item['code'].split('###')[1:]
        messages = []
        user_message = ''
        for j in range(0, len(turn)-1):
            # messages.append({"role": "user", "content": turn[j].strip()})
            user_message += turn[j].strip()
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": turn[-1].strip()})
        instantce = {"conversation_id": i, "system": system, "tools": [json.dumps(item['api_data'])], "messages": messages}
        re['instances'].append(instantce)

    with open(f'data/gorilla_apibench/{ds}.json', 'w') as f:
        json.dump(re, f, indent=4)

for k in dss:
    if 'huggingface' in k:
        api_name = 'huggingface_api'
    elif 'tensorflow' in k:
        api_name = 'tensorflowhub_api'
    elif 'torchhub' in k:
        api_name = 'torchhub_api'
    try:
        process_api(api_name, k)
    except:
        continue

  0%|          | 0/8191 [00:00<?, ?it/s]

100%|██████████| 186/186 [00:00<00:00, 76952.12it/s]


In [26]:
# process gorilla openfunctions-v1
gorilla_openfunctions_v1 = []
with jsonlines.open('gorilla/openfunctions/openfunctions-v1/gorilla_openfunctions_v1_train.json', 'r') as f:
    for line in f:
        gorilla_openfunctions_v1.append(line)

re = {"type": "conversation", "instances": []}

for i, item in enumerate(tqdm(gorilla_openfunctions_v1)):
    system = ''
    messages = [{"role": "user", "content": item['Instruction']}, {"role": "assistant", "content": '\n'.join(item['Output'])}]
    instantce = {"conversation_id": i, "system": system, "tools": item['Functions'], "messages": messages}
    re['instances'].append(instantce)

with open('data/gorilla_openfunctions_v1_train.json', 'w') as f:
    json.dump(re, f, indent=4)

100%|██████████| 12125/12125 [00:00<00:00, 318843.99it/s]


In [30]:
# process ise-uiuc/Magicoder-OSS-Instruct-75K
ds = load_dataset('ise-uiuc/Magicoder-OSS-Instruct-75K')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = ''
    messages = [{"role": "user", "content": item['problem']}, {"role": "assistant", "content": item['solution']}]
    instantce = {"conversation_id": i, "system": system, "messages": messages}
    re['instances'].append(instantce)

with open('data/Magicoder-OSS-Instruct-75K.json', 'w') as f:
    json.dump(re, f, indent=4)

100%|██████████| 75197/75197 [00:03<00:00, 22049.76it/s]


In [32]:
# process RLHFlow/CodeUltraFeedback-standard
ds = load_dataset('RLHFlow/CodeUltraFeedback-standard')
re = {"type": "conversation", "instances": []}
last_chosen = None
for i, item in enumerate(tqdm(ds['train'])):
    system = ''
    messages = item['chosen']
    if last_chosen and messages == last_chosen:
        continue
    last_chosen = messages
    instantce = {"conversation_id": i, "system": system, "messages": messages}
    re['instances'].append(instantce)

with open('data/CodeUltraFeedback-standard.json', 'w') as f:
    json.dump(re, f, indent=4)

Using the latest cached version of the dataset since RLHFlow/CodeUltraFeedback-standard couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/yaojiarui/.cache/huggingface/datasets/RLHFlow___code_ultra_feedback-standard/default/0.0.0/bcb5b923b9d02e6da2b410768fe22f91dfc450db (last modified on Thu Aug  8 18:06:49 2024).
100%|██████████| 50156/50156 [00:02<00:00, 19835.57it/s]


In [4]:
# process meta-math/MetaMathQA
ds = load_dataset('meta-math/MetaMathQA')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = ''
    messages = [{"role": "user", "content": item['query']}, {"role": "assistant", "content": item['response']}]
    instantce = {"conversation_id": i, "system": system, "messages": messages}
    re['instances'].append(instantce)

with open('data/MetaMathQA.json', 'w') as f:
    json.dump(re, f, indent=4)

Generating train split: 100%|██████████| 395000/395000 [00:09<00:00, 42574.12 examples/s]


In [15]:
# process openai/gsm8k
ds = load_dataset('openai/gsm8k', 'main')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = ''
    messages = [{"role": "user", "content": item['question']}, {"role": "assistant", "content": item['answer']}]
    instantce = {"conversation_id": i, "system": system, "messages": messages}
    re['instances'].append(instantce)

with open('data/gsm8k.json', 'w') as f:
    json.dump(re, f, indent=4)

In [6]:
# process camel-ai/math
ds = load_dataset('camel-ai/math')
re = {"type": "conversation", "instances": []}
for i, item in enumerate(tqdm(ds['train'])):
    system = ''
    messages = [{"role": "user", "content": item['message_1']}, {"role": "assistant", "content": item['message_2']}]
    instantce = {"conversation_id": i, "system": system, "messages": messages}
    re['instances'].append(instantce)

with open('data/camel-ai-math.json', 'w') as f:
    json.dump(re, f, indent=4)

In [12]:
# process TIGER-Lab/MathInstruct
ds = load_dataset('TIGER-Lab/MathInstruct')

In [3]:
# process gen-bfcl-filtered
data = []
with jsonlines.open('data/autoGen/gen-bfcl-filtered.json', 'r') as f:
    for line in f:
        data.append(line)
re = {"type": "conversation", "instances": []}
ids = 0
for i, item in enumerate(tqdm(data)):
    for dic in item:
        system = ''
        messages = [{"role": "user", "content": dic['instruction']}, {"role": "assistant", "content": dic['output']['code']+'\n\n'+dic['output']['explanation']}]
        instantce = {"conversation_id": ids, "system": system, "tools": [str(dic['api_data'])], "messages": messages}
        re['instances'].append(instantce)
        ids += 1

with open('data/autoGen/gen-bfcl-filtered.json', 'w') as f:
    json.dump(re, f, indent=4)

100%|██████████| 1726/1726 [00:00<00:00, 24064.89it/s]


In [17]:
ds = load_dataset('codeparrot/apps')

Generating train split: 5000 examples [00:00, 7050.91 examples/s]
Generating test split: 5000 examples [00:07, 635.31 examples/s] 
